In [1]:
import cv2
import numpy as np
import skimage
import imageio
import mpmath
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
from skimage.io import imread, imshow

from skimage.filters import prewitt_v, prewitt_h, laplace, threshold_otsu
from scipy.stats import multivariate_normal
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from tqdm import tqdm
from sklearn.metrics import roc_curve

Feature extractor

In [2]:

def getFeatures(training_img, mask, show_plot=True):
    binary_mask = mask>0.5
    #plt.imshow(binary_mask)
    
    #add dimensions
    # print(binary_mask.shape)
    hsv_training_img = cv2.cvtColor(training_img, cv2.COLOR_BGR2RGB)
    v,s,h = cv2.split(hsv_training_img)
    h, s,v = h*binary_mask, s*binary_mask, v*binary_mask
    # print(h.shape)
    b,g,r = cv2.split(training_img)
    r,g,b = r*binary_mask, g*binary_mask, b*binary_mask

    # get vertical prewitt for separated channels
    vert_prewitt_r = prewitt_v(image=r)
    vert_prewitt_g = prewitt_v(image=g)
    vert_prewitt_b = prewitt_v(image=b)

    # get horizontal prewitt for separated channels
    hori_prewitt_r = prewitt_h(image=r)
    hori_prewitt_g = prewitt_h(image=g)
    hori_prewitt_b = prewitt_h(image=b)

    # get Laplacian for separated channels
    laplace_r = laplace(image=r)
    laplace_g = laplace(image=g)
    laplace_b = laplace(image=b)


    if show_plot:
        # vertical prewitt plot 
        fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize=(16,4))
        plt.subplot(1,3,1), plt.imshow( vert_prewitt_r,cmap="gray"), plt.axis("off")
        plt.subplot(1,3,2), plt.imshow( vert_prewitt_g,cmap="gray"), plt.axis("off")
        plt.subplot(1,3,3), plt.imshow( vert_prewitt_b,cmap="gray"), plt.axis("off")
        plt.suptitle("Vertical Prewitt of RGB image")
        plt.show()

        # horizontal prewitt plot
        fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize=(16,4))
        plt.subplot(1,3,1), plt.imshow( hori_prewitt_r,cmap="gray"), plt.axis("off")
        plt.subplot(1,3,2), plt.imshow( hori_prewitt_g,cmap="gray"), plt.axis("off")
        plt.subplot(1,3,3), plt.imshow( hori_prewitt_b,cmap="gray"), plt.axis("off")
        plt.suptitle("Horizontal Prewitt of RGB image")
        plt.show()

        # laplace plot
        fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize=(16,4))
        plt.subplot(1,3,1), plt.imshow( laplace_r,cmap="gray"), plt.axis("off")
        plt.subplot(1,3,2), plt.imshow( laplace_g,cmap="gray"), plt.axis("off")
        plt.subplot(1,3,3), plt.imshow( laplace_b,cmap="gray"), plt.axis("off")
        plt.suptitle("Laplacian of RGB image")
        plt.show()

    features = [
        vert_prewitt_r, hori_prewitt_r,
        vert_prewitt_g, hori_prewitt_g,
        vert_prewitt_b, hori_prewitt_b,
        laplace_r, laplace_g, laplace_b,
        r, g, b,
        h, s, v
    ]

    flattened_features = np.array([f[binary_mask].flatten() for f in features])
    print(flattened_features[0].shape)

    return np.array(flattened_features)

In [ ]:


# NOTE pass the mask now
def foreground_given_pixel(x,fg_mean, fg_cov, bg_mean, bg_cov,mask):
    N_fg = np.sum(mask)
    N_bg = 270_000 - N_fg
    numerator = multivariate_normal.pdf( x, mean = fg_mean, cov= fg_cov, allow_singular=True) * (N_fg)
    denominator = multivariate_normal.pdf(x, mean=fg_mean, cov=fg_cov, allow_singular=True)*N_fg \
                + multivariate_normal.pdf( x, mean= bg_mean, cov= bg_cov, allow_singular=True) * (N_bg)
    probability = numerator/denominator
    return probability

In [ ]:
def getPixelFeatures(features,row, col):
    pixel_coordinate = row*600 + col
    pixel_features= []
    for f in features:
        pixel_features.append(f[pixel_coordinate])
    return pixel_features

In [3]:
original = imread("Images/image-35.jpg")

mask = imread("Images/mask-35.png", as_gray=True)
#threshold mask
mask = (mask>0.5).astype(int)
# print(np.unique(mask))
identity = np.ones_like(mask,int)
inverse_mask = (mask <=0.5).astype(int)

# feature extraction
# get the feature for fg and bg
original_features = getFeatures(original, identity, show_plot=False)
# print(original_features.shape)
fg_features = getFeatures(original, mask, show_plot=False)
# print(fg_features[0].shape)
bg_features = getFeatures(original, inverse_mask, show_plot=False)
# print(bg_features[0].shape)

# pixel as row with features in the col indices
original_feature_matrix = np.stack(original_features, axis=-1)
# print(original_feature_matrix.shape)

fg_feature_matrix = np.stack(fg_features, axis=-1)
# print(fg_feature_matrix.shape)
fg_mean_vector = np.mean(fg_feature_matrix, axis=0)
fg_cov_matrix = np.cov(fg_feature_matrix, rowvar=False)
# print(fg_mean_vector.shape)
# print(fg_cov_matrix.shape)
bg_feature_matrix = np.stack(bg_features, axis=-1)
bg_mean_vector = np.mean(bg_feature_matrix, axis=0)
bg_cov_matrix = np.cov(bg_feature_matrix, rowvar=False)

#======= Validation image ======
validation_img = imread("Images/image-83.jpg")
# this is only used for the accuracy measure since it represents the foreground
validation_mask = imread("Images/mask-83.png", as_gray=True)
# use null mask since we want to classify evey pixel in the validation image
validation_features  = getFeatures(validation_img, null, show_plot=False)


(270000,)
(74573,)
(195427,)
